<a href="https://colab.research.google.com/github/KarenPHS/News-Classificatoin-BERT-in-Colab-with-TF2-/blob/master/kaggle_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab 設定

In [0]:
from google.colab import files
files.upload()
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [16]:
!kaggle datasets download -d clmentbisaillon/fake-and-real-news-dataset
!unzip fake-and-real-news-dataset.zip

fake-and-real-news-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  fake-and-real-news-dataset.zip
replace Fake.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# 套件安裝

In [17]:
!pip install tqdm  >> /dev/null
!pip install keras-bert

In [18]:
import tensorflow as tf
tf.__version__

'2.2.0-rc4'

In [0]:
!pip install bert-for-tf2 >> /dev/null

# 模型準備

In [0]:
!wget  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip uncased_L-12_H-768_A-12.zip

--2020-05-08 09:51:39--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.203.128, 2404:6800:4008:c04::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip.1’

uncased_L-12_H-768_ 100%[===================>] 388.84M  97.7MB/s    in 4.1s    

2020-05-08 09:51:43 (95.9 MB/s) - ‘uncased_L-12_H-768_A-12.zip.1’ saved [407727028/407727028]

Archive:  uncased_L-12_H-768_A-12.zip
replace uncased_L-12_H-768_A-12/bert_model.ckpt.meta? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# 資料匯入

In [0]:
#事前準備
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [0]:
from keras_bert import Tokenizer
from keras_bert import load_trained_model_from_checkpoint, load_vocabulary
from tqdm import tqdm 
import os
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

max_seq_len = 128
LR = 1e-4
pretrained_path = 'uncased_L-12_H-768_A-12/'
config_path = os.path.join(pretrained_path, 'bert_config.json')  # 模型配置
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt') # 參數
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

token_dict = load_vocabulary(vocab_path)
tokenizer = Tokenizer(token_dict)


def convert_data(df,DATA_COLUMN_1,DATA_COLUMN_2,max_seq_len): # 將要X,Y 轉換為array
    global tokenizer
    indices = []
    for i in tqdm(range(len(df))):
        ids_x, segments_x = tokenizer.encode(df[DATA_COLUMN_1].iloc[i], max_len=max_seq_len)
        indices.append(ids_x)
    indices = np.array(indices)
    predictor = df[DATA_COLUMN_2].to_numpy()
    return [indices, predictor]

def cleansing(x):
    quoteRemoval = x.replace('"','')
    spaceRemoval = re.sub("\s\s+" , " ", quoteRemoval)
    stringRemoval = spaceRemoval.strip()
    urlRemove = re.sub(r'http\S+', '', stringRemoval)
    contract = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in urlRemove.split()]) 
    specialChar = re.sub(r"[^a-zA-Z]+", ' ',urlRemove) 
    return specialChar

def combine(df_true, df_false, ratio):
  df_true['prediction'] = 1
  df_false['prediction'] = 0
  Fake_news_train, Fake_news_test = train_test_split(Fake_news, test_size=ratio)
  True_news_train, True_news_test = train_test_split(True_news, test_size=ratio)
  train = pd.concat([Fake_news_train, True_news_train],axis=0)
  test = pd.concat([Fake_news_test, True_news_test],axis=0)
  train = shuffle(train)
  test = shuffle(test)
  return train, test

# start
Fake_news = pd.read_csv("Fake.csv")
True_news = pd.read_csv("True.csv")
train, test = combine(True_news, Fake_news, 0.3)

train['title'] = train['title'].apply(cleansing).str.lower()
train['text'] = train['text'].apply(cleansing).str.lower()
test['title'] = test['title'].apply(cleansing).str.lower()
test['text'] = test['text'].apply(cleansing).str.lower()

X_train, Y_train = convert_data(train, 'text', 'prediction',max_seq_len)
X_test, Y_test = convert_data(test, 'text', 'prediction',max_seq_len)

train = tf.data.Dataset.from_tensor_slices((dX_train, Y_train))
test = tf.data.Dataset.from_tensor_slices((X_test, Y_test))


Using TensorFlow backend.
100%|██████████| 13471/13471 [01:08<00:00, 195.74it/s]


In [0]:
# 查東西用 tokenizer.decode(X_train[2].tolist())


# 製作模型

In [0]:
import bert
from bert import BertModelLayer
from bert.loader import StockBertConfig, map_stock_config_to_params, load_stock_weights
from bert.tokenization.bert_tokenization import FullTokenizer

def flatten_layers(root_layer):
    if isinstance(root_layer, keras.layers.Layer):
        yield root_layer
    for layer in root_layer._layers:
        for sub_layer in flatten_layers(layer):
            yield sub_layer


def freeze_bert_layers(l_bert):
    """
    Freezes all but LayerNorm and adapter layers - see arXiv:1902.00751.
    """
    for layer in flatten_layers(l_bert):
        if layer.name in ["LayerNorm", "adapter-down", "adapter-up"]:
            layer.trainable = True
        elif len(layer._layers) == 0:
            layer.trainable = False
        l_bert.embeddings_layer.trainable = False


def create_learning_rate_scheduler(max_learn_rate=5e-5,
                                   end_learn_rate=1e-7,
                                   warmup_epoch_count=10,
                                   total_epoch_count=90):

    def lr_scheduler(epoch):
        if epoch < warmup_epoch_count:
            res = (max_learn_rate/warmup_epoch_count) * (epoch + 1)
        else:
            res = max_learn_rate*math.exp(math.log(end_learn_rate/max_learn_rate)*(epoch-warmup_epoch_count+1)/(total_epoch_count-warmup_epoch_count+1))
        return float(res)
    learning_rate_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_scheduler, verbose=1)

    return learning_rate_scheduler


## Creating a model

In [0]:
from tensorflow import keras
def create_model(max_seq_len, adapter_size=64):
  """Creates a classification model."""

  #adapter_size = 64  # see - arXiv:1902.00751
  bert_model_name="uncased_L-12_H-768_A-12"
  bert_ckpt_dir    = bert_model_name
  bert_ckpt_file   = os.path.join(bert_ckpt_dir, "bert_model.ckpt")
  bert_config_file = os.path.join(bert_ckpt_dir, "bert_config.json")

  # create the bert layer
  with tf.io.gfile.GFile(bert_config_file, "r") as reader:
      bc = StockBertConfig.from_json_string(reader.read())
      bert_params = map_stock_config_to_params(bc)
      bert_params.adapter_size = adapter_size
      bert = BertModelLayer.from_params(bert_params, name="bert")
        
  input_ids      = keras.layers.Input(shape=(max_seq_len,), dtype='int32', name="input_ids")
  # token_type_ids = keras.layers.Input(shape=(max_seq_len,), dtype='int32', name="token_type_ids")
  # output         = bert([input_ids, token_type_ids])
  output         = bert(input_ids)

  print("bert shape", output.shape)
  cls_out = keras.layers.Lambda(lambda seq: seq[:, 0, :])(output)
  cls_out = keras.layers.Dropout(0.5)(cls_out)
  logits = keras.layers.Dense(units=768, activation="tanh")(cls_out)
  logits = keras.layers.Dropout(0.5)(logits)
  logits = keras.layers.Dense(units=2, activation="softmax")(logits)

  # model = keras.Model(inputs=[input_ids, token_type_ids], outputs=logits)
  # model.build(input_shape=[(None, max_seq_len), (None, max_seq_len)])
  model = keras.Model(inputs=input_ids, outputs=logits)
  model.build(input_shape=(None, max_seq_len))

  # load the pre-trained model weights
  load_stock_weights(bert, bert_ckpt_file)

  # freeze weights if adapter-BERT is used
  if adapter_size is not None:
      freeze_bert_layers(bert)

  model.compile(optimizer=keras.optimizers.Adam(),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")])

  model.summary()
        
  return model

In [0]:
adapter_size = None # use None to fine-tune all of BERT
max_seq_len = 128
model = create_model(max_seq_len, adapter_size=adapter_size)

bert shape (None, 128, 768)
Done loading 196 BERT weights from: uncased_L-12_H-768_A-12/bert_model.ckpt into <bert.model.BertModelLayer object at 0x7f36aae739b0> (prefix:bert). Count of weights not found in the checkpoint was: [0]. Count of weights with mismatched shape: [0]
Unused weights from checkpoint: 
	bert/embeddings/token_type_embeddings
	bert/pooler/dense/bias
	bert/pooler/dense/kernel
	cls/predictions/output_bias
	cls/predictions/transform/LayerNorm/beta
	cls/predictions/transform/LayerNorm/gamma
	cls/predictions/transform/dense/bias
	cls/predictions/transform/dense/kernel
	cls/seq_relationship/output_bias
	cls/seq_relationship/output_weights
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 128)]             0         
_________________________________________________________________
bert (BertModelLayer)        (None, 128, 768)          10889

In [0]:
%%time
import datetime

log_dir = ".log/movie_reviews/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%s")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir)

total_epoch_count = 20
# model.fit(x=(data.train_x, data.train_x_token_types), y=data.train_y,
model.fit(train,
          validation_split=0.1,
          batch_size=48,
          shuffle=True,
          epochs=total_epoch_count,
          callbacks=[create_learning_rate_scheduler(max_learn_rate=1e-5,
                                                    end_learn_rate=1e-7,
                                                    warmup_epoch_count=20,
                                                    total_epoch_count=total_epoch_count),
                     keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True),
                     tensorboard_callback])

model.save_weights('./news.h5', overwrite=True)


Epoch 00001: LearningRateScheduler reducing learning rate to 5.000000000000001e-07.
Epoch 1/20
590/590 [==============================] - 1501s 3s/step - loss: 0.7072 - acc: 0.5303 - val_loss: 0.6431 - val_acc: 0.7060 - lr: 5.0000e-07

Epoch 00002: LearningRateScheduler reducing learning rate to 1.0000000000000002e-06.
Epoch 2/20
578/590 [============================>.] - ETA: 29s - loss: 0.6637 - acc: 0.5977

KeyboardInterrupt: ignored

In [0]:
%%time

_, train_acc = model.evaluate(train)
_, test_acc = model.evaluate(test)

print("train acc", train_acc)
print(" test acc", test_acc)

421/421 [==============================] - 75s 178ms/step - loss: 0.3145 - acc: 0.9989
train acc 0.9988226890563965
 test acc 0.9988865256309509


MessageError: ignored

## Evaluation

In [0]:

%%time 
max_seq_len = 128
model = create_model(max_seq_len, adapter_size=None)
model.load_weights("news.h5")

_, train_acc = model.evaluate(data.train_x, data.train_y)
_, test_acc = model.evaluate(data.test_x, data.test_y)

print("train acc", train_acc)
print(" test acc", test_acc)

In [0]:
files.download('/content/news.h5')

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 51152, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil